In [32]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

In [38]:
df = pd.read_csv('daily covid - owid.csv', parse_dates=['date'])

df = df.sort_values('date')
df.set_index('date', inplace=True)
df.index = pd.DatetimeIndex(df.index, freq='D')

df['cases'] = np.log(df['cases'].replace(0, 1))

df['cases'] = df['cases'].apply(lambda x: np.nan if x > 20 else x)

df = df.ffill()

train = df.loc['2020-01-01':'2023-09-25']
test = df.loc['2023-09-26':'2024-05-25']

In [39]:
model = ExponentialSmoothing(
    train['cases'], 
    trend='add', 
    seasonal='add', 
    seasonal_periods=7  # Weekly seasonality
).fit(optimized=True, minimize_kwargs={'options': {'maxiter': 1000}})


In [40]:
forecast_start = pd.to_datetime('2024-05-25')  # Starting date for the forecast
forecast_end = pd.to_datetime('2025-05-30')    # Ending date for the forecast
forecast_index = pd.date_range(start=forecast_start, end=forecast_end, freq='D')
future_predictions = model.forecast(len(forecast_index))

future_predictions = np.exp(future_predictions)

In [42]:
forecast_df = pd.DataFrame({'date': forecast_index, 'forecasted_cases': future_predictions})
forecast_df.to_csv('forecasted_cases.csv', index=False)